# 2 Create Mobility Demand

The goal of the following notebook is to compute a realistic Mobility Demand.
___
The Mobility Demand $D = \{T_1, \dots, T_N\}$ is a multiset of $N$ trips (one per each vehicle) within an urban environment. 
A single trip $T_v=(o,d)$ for a vehicle $v$ is defined by its origin location $o$ and destination location $d$.
To compute $D$, we use an origin-destination matrix $M$ where an element $m_{o, d}\in M$ describes the number of vehicles' trips that start in tile $o$ and end in tile $d$. 
Then, we iterate $N$ times the following procedure. 
- A vehicle's $v$ trip is a pair $T_v=(e_o, e_d)$ generated by selecting at random a matrix element $m_{o,d} \in M$ with a probability $p_{o, d} \propto m_{o, d}$ and uniformly at random two edges $e_o, e_d \in E$ within tiles $o$ and $d$, respectively.
___

### Utils

In [ ]:
import warnings
warnings.filterwarnings("ignore")

#import pandas as pd
import numpy as np
import geopandas as gpd
import sumolib
import json
from utils_mobility_demand import create_dict_tile_edges, create_dict_tile_nodes, create_traffic_demand_from_matrix, create_xml_flows
from utils_mobility_demand import pearson_cpc_matrices, invalid_routes_in_demand
import subprocess
import os
from skmob.tessellation import tilers
#import re
#from routing_utils import from_sumo_to_igraph_network

In [ ]:
def compute_p_cpc_choices(od_matrix, choices_list):
    matrix_test = np.zeros((od_matrix.shape))
    for i, j in choices_list:
        matrix_test[i][j]+=1 
        
    return pearson_cpc_matrices(od_matrix, matrix_test)

In [ ]:
def repair_invalid_routes(dict_mobility_demand, road_network_path, threshold_km=1.2, allow_self_tiles=False):
    
    list_invalid_ids = invalid_routes_in_demand(dict_mobility_demand, road_network_path)

    while len(list_invalid_ids)>0:

        print(f"Repair {len(list_invalid_ids)} invalid routes.")

        for id_to_repair in list_invalid_ids:
            
            seed = int(id_to_repair.split("_")[-1])

            print(id_to_repair, seed)

            od_list_repair, _ = create_traffic_demand_from_matrix(od_matrix, dict_mapping, 1, road_network, 
                                                              node_mode=node_mode, threshold_km=threshold_km, 
                                                                  random_seed=seed, allow_self_tiles=allow_self_tiles)

            dict_mobility_demand[id_to_repair]["element"] = od_list_repair[0]


        list_invalid_ids = invalid_routes_in_demand(dict_mobility_demand, road_network_path)
        


In [ ]:
def apply_duarouter(route_file_path, road_network_path, weight, output_demand_filename, seed_duarouter=None, rm_loops="true"):
    

    if seed_duarouter is None:
        np.random.seed()
        seed_duarouter = np.random.randint(0, 1e6)

    options_duarouter = f"--weights.random-factor {weight} --routing-threads 32"
    
    command_str = "duarouter --route-files "+route_file_path+" "+\
        " --net-file "+road_network_path+" "+options_duarouter+\
    " --random false --seed "+str(seed_duarouter)+\
    " --output-file "+output_demand_filename + " --junction-taz true"
            
    # call duarouter process
    p = subprocess.Popen(command_str, shell=True, stdout=subprocess.PIPE, 
                                     stderr=subprocess.STDOUT)
    retval = p.wait()

    # remove .alt file
    os.remove(output_demand_filename.split(".rou")[0]+".rou.alt"+output_demand_filename.split(".rou")[1])

    
    

In [ ]:
def create_dict_set_vehicles(N, n_rep_min, n_rep_max, percentages):

    dict_set_vehicles = {}

    for pct in percentages:
        if pct not in [0, 100]:
            dict_set_vehicles[str(pct)] = {}
            for rep in range(n_rep_min, n_rep_max+1):
                max_ind = int((pct * N)/100)
                permutation_array = np.random.permutation(np.arange(N))[:max_ind]
                # take the pct elements 
                dict_set_vehicles[str(pct)][str(rep)] = [int(x) for x in permutation_array]
                
    return dict_set_vehicles

#### Parameteres

In [ ]:
city = "milan"

In [ ]:
# od matrix path
od_matrix_path = f"../data/od_matrices/od_matrix_{city}.npy"

# tessellation path
shapefile_path = f"../data/bbox_cities/bbox_road_network_{city}.geojson"

# road network path
road_network_path = f"../data/road_networks/sumo_road_network_{city}.net.xml"

# tmp
#road_network_path = f"../data/{city}/road_networks/road_network_{city}_v4.net.xml"


### 1. Load the od-matrix

In [ ]:
od_matrix = np.load(od_matrix_path)

print(f"The OD Matrix describes {int(od_matrix.sum())} trips")
print(f"The OD Matrix contains {len(od_matrix.nonzero()[0])} flows")
print(f"The OD Matrix refers to a tiling of {od_matrix.shape[0]} tiles")

### 2. Load the tessellation

In [ ]:
city_shape = gpd.read_file(shapefile_path)
tile_size_meters = 1000
tessellation = tilers.tiler.get('squared', base_shape=city_shape, meters=tile_size_meters)
print("# tiles:",len(tessellation))

### 3. Load the road network

In [ ]:
road_network = sumolib.net.readNet(road_network_path, withInternal=False)

### 4. Create dict tile to edges and dict tile to nodes

#### 4.1 Tile -> Edges

Associate to each tile the corresponding edges

In [ ]:
dict_tile_edges = create_dict_tile_edges(road_network, tessellation, exclude_roundabouts=True)

#### 4.2 Tile -> Nodes

Associate to each tile the corresponding nodes

In [ ]:
dict_tile_nodes = create_dict_tile_nodes(road_network, tessellation, exclude_roundabouts=True)

### 5 Create the Traffic Starting from the OD-Matrix

This routine creates:
- a mobility demand (origin, destination, departure time) for each of the $N$ vehicles;
- a dictionary that maps for each adoption rate $r$ and repetition the vehicles that will follow the navigation service (routed vehicles).


In [ ]:
# this define if it its node or edge based (We suggest to use Node mode)
dict_mapping = dict_tile_nodes
node_mode = True

vehicle_suffix = "vehicle_"

In [ ]:
%%time

list_N = [5*1000]
list_w = []

# min and max departure time (in seconds)
min_dt, max_dt = 0, 3600

# how many choice of routed vs non-routed vehicles at each adoption rate
n_rep_min, n_rep_max = 0, 19


for N in list_N:
    
    demand_name = f"N{N}"
    
    # create the output folder for the MD
    output_folder_md = f"../data/{city}/mobility_demand/{demand_name}/"
    if not os.path.exists(output_folder_md):
        os.makedirs(output_folder_md, exist_ok=True)
        
    
    # Compute the ODs
    np.random.seed()
    seed_od = np.random.randint(0, 1e7)
    od_list, choices_list = create_traffic_demand_from_matrix(od_matrix, dict_mapping, N, road_network, 
                                                      node_mode=node_mode, threshold_km=1.2, random_seed=seed_od,
                                                      allow_self_tiles=False)

    pearson, cpc = compute_p_cpc_choices(od_matrix, choices_list)
    print("Scores:", pearson, cpc)


    # Create a mobility demand by assigning a dep. time to each OD
    np.random.seed()
    seed_dep_time = np.random.randint(0, 1e7)
    np.random.seed(seed_dep_time)

    dict_mobility_demand = {}
    for ind, trip in enumerate(od_list):
        dep_time = np.random.randint(min_dt, max_dt)
        dict_mobility_demand[vehicle_suffix+str(ind)] = {'element':trip, 
                                                         'time': dep_time,
                                                         'via': False, 'number':1, 'dt':10}

    # Repair invalid routes
    repair_invalid_routes(dict_mobility_demand, road_network_path, threshold_km=1.2, allow_self_tiles=False)


    # create and save the dict_set_vehicles 
    np.random.seed()
    seed_dict = np.random.randint(0, 1e7)
    np.random.seed(seed_dict)
    dict_set_vehicles = create_dict_set_vehicles(N, n_rep_min, n_rep_max, list(np.arange(0, 101, 10)))

    # Save the dict set vehicles
    dict_set_file_path = f"{output_folder_md}dict_set_vehicles_{city}_{demand_name}.json"
    with open(dict_set_file_path, 'w') as json_file:
        json.dump(dict_set_vehicles, json_file)


    # Save the mobility demand into a .xml.rou file
    route_file_path = f"{output_folder_md}/route_file_{city}_{demand_name}.rou.xml"
    create_xml_flows(dict_mobility_demand, filename=route_file_path, node_mode=True, lane_best=True)


    # Save the mobility demand + info into a dictionary (.json)
    dict_w_info = {"parameters":{"N":N, "seed_od": seed_od, 
                                 "seed_dep_time": seed_dep_time, 
                                 "seed_dict":seed_dict,
                                 "pearson":pearson,
                                 "cpc": cpc}, "demand": dict_mobility_demand}

    with open(f"{output_folder_md}dict_mobility_demand_{city}_{demand_name}.json", 'w') as json_file:
        json.dump(dict_w_info, json_file)

    